In [1]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Deep Learning
import torch
import torch.nn            as nn
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer
from torch.optim.lr_scheduler import LRScheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import torchinfo
from torchmetrics.classification import MulticlassAccuracy
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.v2 as v2
import torchvision.models as models


import skimage.io as ski
from sklearn.utils.class_weight import compute_class_weight

# Machine Learning
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import auc, balanced_accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_predict, train_test_split, cross_val_score, ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import torch.optim as optim

# Miscellaneous
import copy
from enum import auto, Enum, unique
import math
import os
from platform import python_version
import random
import time

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import HTML, Image
from IPython.display import display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout, SelectionSlider
from ipywidgets import interact

from os import listdir

import skimage as ski


from DLfunctions import Train_model, Train_modelPROB
# from DeepLearningPyTorch import NNMode, TrainModel, RunEpoch, TrainModelSch, RunEpochSch

In [2]:
import importlib
import DLfunctions  # Replace this with the name of your module

importlib.reload(DLfunctions)

from DLfunctions import Train_model

In [3]:
df_balanced= pd.read_csv(r'df_balanced.csv')

In [4]:
df_balanced

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate
0,0038fd5f09f5,0f0fb8cd3ee9,0b5796d5a3ba,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0038fd5f09f5,0f0fb8cd3ee9,dba7058fdf23,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0038fd5f09f5,0f0fb8cd3ee9,b52f05d70451,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0038fd5f09f5,0f0fb8cd3ee9,524ef86a0c4a,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0038fd5f09f5,0f0fb8cd3ee9,656dbe20aa57,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281995,fffda3f22362,39ca5eaafffe,ef8ea171d478,0,0,0,0,0,0,0,0,0,0,0,0,0,0
281996,fffda3f22362,39ca5eaafffe,e30001ca271a,0,0,0,0,0,0,0,0,0,0,0,0,0,0
281997,fffda3f22362,39ca5eaafffe,2eeac479445f,0,0,0,0,0,0,0,0,0,0,0,0,0,0
281998,fffda3f22362,39ca5eaafffe,472df1c77890,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# Upload CSV

# csv_file_path = r'/media/diana/My Passport/desktop lenovo/train.csv'
csv_file_path = r'/home/diana/train.csv'
df = pd.read_csv(csv_file_path)

# path of data folder
# data_folder_path =  r'/media/diana/My Passport/rsna_jpeg/train-jpegs'
data_folder_path =  r'/home/diana/train-jpegs'


In [6]:
# Set the Device
TORCH_DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #<! for first device: `cuda:0` 
print(f'The chosen device: {TORCH_DEVICE}')

The chosen device: cuda


In [7]:
#Filter patients with between 200 to 250 images
image_counts = df.groupby('StudyInstanceUID')['SOPInstanceUID'].count()
filtered_patients = image_counts[(image_counts >= 30) & (image_counts <= 1000)]
filtered_patient_ids = filtered_patients.index
filtered_df = df[df['StudyInstanceUID'].isin(filtered_patient_ids)]
patient_status_counts = filtered_df.drop_duplicates(subset='StudyInstanceUID')['negative_exam_for_pe'].value_counts()

print(f"Number of patients with 200 to 250 images: {len(filtered_patient_ids)}")
print(f"Number of healthy patients: {patient_status_counts.get(1, 0)}")
print(f"Number of sick patients: {patient_status_counts.get(0, 0)}")

Number of patients with 200 to 250 images: 7276
Number of healthy patients: 4910
Number of sick patients: 2366


In [8]:
#Balance Data

## all data - taking half of the neg and all pos (using JPGs)
# delete duplicates - StudyInstanceUID
unique_patients_df = filtered_df.drop_duplicates(subset='StudyInstanceUID')
patient_counts = unique_patients_df['negative_exam_for_pe'].value_counts()
balanced_unique = filtered_df.drop_duplicates(subset='StudyInstanceUID')

#num of neg to take as num of positives
neg_bal_patients = balanced_unique[balanced_unique['negative_exam_for_pe'] == 1].head(patient_counts[0])
#num of pos - all pos
pos_bal_patients = balanced_unique[balanced_unique['negative_exam_for_pe'] == 0]

neg_bal_patients_ids = neg_bal_patients['StudyInstanceUID'].tolist()
pos_bal_patients_ids = pos_bal_patients['StudyInstanceUID'].tolist()

print(f"num of patients without PE: {len(neg_bal_patients)}")
print(f"num of patients with pe: {len(pos_bal_patients)}")

#df
balanced_patients=pd.concat([neg_bal_patients, pos_bal_patients], ignore_index=True)
#list of all IDs
ids_balanced=neg_bal_patients_ids + pos_bal_patients_ids
# extracting from original csv to a new working file


# df_balanced=filtered_df[filtered_df['StudyInstanceUID'].isin(ids_balanced)]

num of patients without PE: 2366
num of patients with pe: 2366


## 2D

In [9]:

# def select_10_images_per_patient(df_balanced, root_dir):
 
#     # List to store the selected rows
#     selected_rows = []

#     # Group by StudyInstanceUID (Patient ID)
#     grouped = df_balanced.groupby('StudyInstanceUID')

#     # Loop over each patient group
#     for patient_id, group in grouped:
#         group_series = group['SeriesInstanceUID'].unique()[0]  # Get the unique series for the patient

#         # Path to the patient's image directory
#         path = os.path.join(root_dir, patient_id, group_series)

#         # Get the image filenames from the directory
#         slices = os.listdir(path)

#         # Filter and sort the filenames by the position part (e.g., 001_SERIAL_NUMBER)
#         valid_slices = []
#         for s in slices:
#             try:
#                 # Extract the numeric part before the first underscore
#                 position = int(s.split('_')[0])  # Extracts the "001" from "001_SERIAL_NUMBER"
#                 valid_slices.append((position, s))
#             except ValueError:
#                 # Skip files that don't follow the expected pattern
#                 continue

#         # Sort the valid slices by the numeric position
#         valid_slices.sort(key=lambda x: x[0])

#         # Extract just the filenames (sorted by position)
#         sorted_slices = [s[1] for s in valid_slices]

#         # Calculate the number of images and the 55% starting index
#         num_images = len(sorted_slices)
#         start_idx = int(num_images * 0.50)

#         # Select exactly 10 images starting from the 55% index
#         selected_slices = sorted_slices[start_idx:start_idx + 20]

#         # Print selected slices for debugging
#         print(f"Selected slices: {selected_slices}")

#         # Strip the serial number and .jpg extension from selected_slices to get the SOPInstanceUID
#         db_selected_slices = [s.split('_')[1].replace('.jpg', '') for s in selected_slices]

#         # Print the cleaned SOPInstanceUIDs for debugging
#         print(f"DB selected SOPInstanceUIDs: {db_selected_slices}")

#         # Filter the group to keep only the rows where SOPInstanceUID matches the cleaned filenames
#         selected_group = group[group['SOPInstanceUID'].isin(db_selected_slices)]

#         # Append these rows to the list
#         selected_rows.append(selected_group)

#     # Concatenate all the selected rows into a new DataFrame
#     new_df = pd.concat(selected_rows).reset_index(drop=True)

#     return new_df


# # Create a new DataFrame with only 10 images per patient
# df_balanced = select_10_images_per_patient(df_balanced, root_dir=data_folder_path) #(filtered_df, root_dir=data_folder_path)



In [10]:
df_balanced

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate
0,0038fd5f09f5,0f0fb8cd3ee9,0b5796d5a3ba,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0038fd5f09f5,0f0fb8cd3ee9,dba7058fdf23,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0038fd5f09f5,0f0fb8cd3ee9,b52f05d70451,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0038fd5f09f5,0f0fb8cd3ee9,524ef86a0c4a,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0038fd5f09f5,0f0fb8cd3ee9,656dbe20aa57,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281995,fffda3f22362,39ca5eaafffe,ef8ea171d478,0,0,0,0,0,0,0,0,0,0,0,0,0,0
281996,fffda3f22362,39ca5eaafffe,e30001ca271a,0,0,0,0,0,0,0,0,0,0,0,0,0,0
281997,fffda3f22362,39ca5eaafffe,2eeac479445f,0,0,0,0,0,0,0,0,0,0,0,0,0,0
281998,fffda3f22362,39ca5eaafffe,472df1c77890,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### fixing the valuse so 1 is positive and 0 negative

In [11]:
# Flipping the values in the 'negative_exam_for_pe' column
df_balanced['negative_exam_for_pe'] = df_balanced['negative_exam_for_pe'].apply(lambda x: 1 if x == 0 else 0)


In [12]:
df_balanced['negative_exam_for_pe'].value_counts()

negative_exam_for_pe
1    141000
0    141000
Name: count, dtype: int64

In [13]:
class CustomTrainDataset(Dataset):
    def __init__(self, annotations, root_dir, device=TORCH_DEVICE): 
        self.annotations = annotations
        self.root_dir = root_dir
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        

        # Group by patient ID
        self.groups = self.annotations.groupby('StudyInstanceUID')
        self.patient_ids =list(self.groups.groups.keys())
        

    def __len__(self):
         return len(self.annotations)

    def load_image(self, patient_id, idx):
        
        #get patient data
        group_study = self.groups.get_group(patient_id) #patient id
        group_series = (group_study['SeriesInstanceUID'].unique())[0] #series id
        
        #path of patient files
        path=os.path.join(self.root_dir, patient_id, group_series)
        
        #file list of patients
        patient_files = os.listdir(path)
        
        patient_file_names = [f.split('_')[-1].split('.')[0].strip() for f in patient_files]
        
        #find the correct file in file list
        i_idx = patient_file_names.index(self.annotations['SOPInstanceUID'].iloc[idx])
        
        #path of image
        path=os.path.join(self.root_dir, patient_id, group_series, patient_files[i_idx])
        
        #reading image
        image = ski.io.imread(path)
        # image = imread(path)
        
        
        # Check if the image has only one channel, convert it to 3 channels
        if len(image.shape) == 2 or image.shape[2] == 1:
            image = np.stack([image] * 3, axis=-1)

        # Normalize    
        imgSize = 224
        # imgSize = 160
        vMean = np.array([0.485, 0.456, 0.406])
        vStd  = np.array([0.229, 0.224, 0.225])

        oPreProcess = v2.Compose([
            v2.ToImage(), 
            v2.RandomHorizontalFlip(),
            v2.RandomVerticalFlip(),
            v2.RandomRotation(degrees=15),
            v2.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
            v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            v2.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
            v2.Resize(imgSize),
            v2.CenterCrop(imgSize),
            v2.ToDtype(torch.float32, scale=True),  
            v2.Normalize(mean=vMean, std=vStd),  
        ])

        image = oPreProcess(image)
        image = image.to(self.device)
        
        return image
    
    def __getitem__(self,idx):
        
        idx = int(idx)
        
        # patient num
        patient_id = self.annotations['StudyInstanceUID'].iloc[idx]
        
        # Use the load_slices method for image loading and sampling
        image = self.load_image(patient_id, idx) 

        # Split into three separate 1-channel images
        image1 = image[0:1, :, :]  # Extract the first channel
        image2 = image[1:2, :, :]  # Extract the second channel
        image3 = image[2:3, :, :]  # Extract the third channel        
        
        image1 = image1.to(self.device)
        image2 = image2.to(self.device)
        image3 = image3.to(self.device)           

        label = self.annotations['negative_exam_for_pe'].iloc[idx]
        label = torch.tensor(label, dtype=torch.float32).unsqueeze(0).to(self.device)
        
        return (image1, image2, image3), label
        # return image, label 

In [14]:
class CustomValDataset(Dataset):
    def __init__(self, annotations, root_dir, device=TORCH_DEVICE): 
        self.annotations = annotations
        self.root_dir = root_dir
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        

        # Group by patient ID
        self.groups = self.annotations.groupby('StudyInstanceUID')
        self.patient_ids =list(self.groups.groups.keys())
        

    def __len__(self):
         return len(self.annotations)

    def load_image(self, patient_id, idx):
        
        #get patient data
        group_study = self.groups.get_group(patient_id) #patient id
        group_series = (group_study['SeriesInstanceUID'].unique())[0] #series id
        
        #path of patient files
        path=os.path.join(self.root_dir, patient_id, group_series)
        
        #file list of patients
        patient_files = os.listdir(path)
        
        patient_file_names = [f.split('_')[-1].split('.')[0].strip() for f in patient_files]
        
        #find the correct file in file list
        i_idx = patient_file_names.index(self.annotations['SOPInstanceUID'].iloc[idx])
        
        #path of image
        path=os.path.join(self.root_dir, patient_id, group_series, patient_files[i_idx])
        
        #reading image
        image = ski.io.imread(path)
        # image = imread(path)
        
        
        # Check if the image has only one channel, convert it to 3 channels
        if len(image.shape) == 2 or image.shape[2] == 1:
            image = np.stack([image] * 3, axis=-1)

        # Normalize    
        imgSize = 224
        # imgSize = 160
        vMean = np.array([0.485, 0.456, 0.406])
        vStd  = np.array([0.229, 0.224, 0.225])

        oPreProcess = v2.Compose([
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(mean = vMean,std=vStd),
        ])

        image = oPreProcess(image)
        image = image.to(self.device)
        
        return image
    
    def __getitem__(self,idx):
        
        idx = int(idx)
        
        # patient num
        patient_id = self.annotations['StudyInstanceUID'].iloc[idx]
        
        # Use the load_slices method for image loading and sampling
        image = self.load_image(patient_id, idx)  

        # Split into three separate 1-channel images
        image1 = image[0:1, :, :]  # Extract the first channel
        image2 = image[1:2, :, :]  # Extract the second channel
        image3 = image[2:3, :, :]  # Extract the third channel  

        image1 = image1.to(self.device)
        image2 = image2.to(self.device)
        image3 = image3.to(self.device)
                     
                
        label = self.annotations['negative_exam_for_pe'].iloc[idx]
        label = torch.tensor(label, dtype=torch.float32).unsqueeze(0).to(self.device)
     

        return (image1, image2, image3), label
        # return image, label 

In [15]:
### Split train test 

# Split train test according to patient
group_splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get the indices for train and test sets based on groups
train_index, val_index = next(group_splitter.split(df_balanced, df_balanced['negative_exam_for_pe'], groups=df_balanced['StudyInstanceUID']))

#create train and test df's based on the indices
train_df, val_df = df_balanced.iloc[train_index,:], df_balanced.iloc[val_index,:]

In [16]:
print(f'Training data size: {len(train_df)}')
print(f'Testing data size: {len(val_df)}')

print(f"Training num of patients: {len(train_df['StudyInstanceUID'].unique())}")
print(f"Testing num of patients: {len(val_df['StudyInstanceUID'].unique())}")

print(f'Classes in train: {train_df["negative_exam_for_pe"].unique()}')
print(f'Classes in test: {val_df["negative_exam_for_pe"].unique()}')

Training data size: 225600
Testing data size: 56400
Training num of patients: 3760
Testing num of patients: 940
Classes in train: [1 0]
Classes in test: [1 0]


In [17]:
# Creat Datasets
train_dataset = CustomTrainDataset(annotations=train_df, root_dir=data_folder_path)
val_dataset = CustomValDataset(annotations=val_df, root_dir=data_folder_path)
# Check Data
print(f'Training data size: {len(train_dataset)}')
print(f'Testing data size: {len(val_dataset)}')

Training data size: 225600
Testing data size: 56400


In [18]:
# DataLoaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last= True)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)


In [19]:
#1st batch of train dataset

train_images, train_labels = next(iter(train_loader)) #<! PyTorch Tensors

print(f'The number of tensors in the batch: {len(train_images)}')
print(f'Shape of the first channel tensor: {train_images[0].shape}')
print(f'Shape of the second channel tensor: {train_images[1].shape}')
print(f'Shape of the third channel tensor: {train_images[2].shape}')

# print(f'The batch features dimensions: {train_images.shape}')
print(f'The batch labels dimensions: {train_labels.shape}')
print(f'The batch labels unique values: {train_labels.unique()}')

The number of tensors in the batch: 3
Shape of the first channel tensor: torch.Size([32, 1, 224, 224])
Shape of the second channel tensor: torch.Size([32, 1, 224, 224])
Shape of the third channel tensor: torch.Size([32, 1, 224, 224])
The batch labels dimensions: torch.Size([32, 1])
The batch labels unique values: tensor([0., 1.], device='cuda:0')


In [20]:
#1st batch of test dataset

test_images, test_labels = next(iter(test_loader)) #<! PyTorch Tensors

print(f'The number of tensors in the batch: {len(train_images)}')
print(f'Shape of the first channel tensor: {test_images[0].shape}')
print(f'Shape of the second channel tensor: {test_images[1].shape}')
print(f'Shape of the third channel tensor: {test_images[2].shape}')

# print(f'The batch features dimensions: {test_images.shape}')
print(f'The batch labels dimensions: {test_labels.shape}')
print(f'The batch labels unique values: {test_labels.unique()}')

The number of tensors in the batch: 3
Shape of the first channel tensor: torch.Size([32, 1, 256, 256])
Shape of the second channel tensor: torch.Size([32, 1, 256, 256])
Shape of the third channel tensor: torch.Size([32, 1, 256, 256])
The batch labels dimensions: torch.Size([32, 1])
The batch labels unique values: tensor([1.], device='cuda:0')


In [25]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, dropout_rate=0.5):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        
        # Initialize downsample if needed
        self.downsample = downsample

    def forward(self, x):
        identity = x
        
        if self.downsample is not None:
            identity = self.downsample(x)  # Adjust shortcut connection if downsampling is required

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.dropout(out)

        out += identity  # Add the residual (shortcut) connection
        out = self.relu(out)

        return out

class CustomConvNetWithResiduals(nn.Module):
    def __init__(self, in_channels, num_classes=1, dropout_rate=0.5):
        super(CustomConvNetWithResiduals, self).__init__()

        # Initial convolutional layer
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(256)

        self.relu = nn.ReLU(inplace=True)
        # self.relu = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Create downsample layers for residual blocks
        self.residual_block1_downsample = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(512)
        )
        
        self.residual_block2_downsample = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(1024)
        )

        # Residual blocks
        self.residual_block1 = ResidualBlock(256, 512, stride=2, downsample=self.residual_block1_downsample, dropout_rate=dropout_rate)
        self.residual_block2 = ResidualBlock(512, 1024, stride=2, downsample=self.residual_block2_downsample, dropout_rate=dropout_rate)

        # Global average pooling and fully connected layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Pass through initial convolutional layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)

        x = self.dropout(x)
        x = self.maxpool(x)

        # Pass through the residual blocks
        x = self.residual_block1(x)
        x = self.residual_block2(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x

# def GetModel(num_classes=1, dropout_rate=0.5) -> nn.Module:
#     return CustomConvNetWithResiduals(num_classes=num_classes, dropout_rate=dropout_rate)


In [26]:
class SiameseNetwork(nn.Module):
    def __init__(self, num_classes=1, dropout_rate=0.5):
        super(SiameseNetwork, self).__init__()
        
        # Three identical networks for the three channels
        self.branch1 = CustomConvNetWithResiduals(in_channels=1, num_classes=512, dropout_rate=dropout_rate)
        self.branch2 = CustomConvNetWithResiduals(in_channels=1, num_classes=512, dropout_rate=dropout_rate)
        self.branch3 = CustomConvNetWithResiduals(in_channels=1, num_classes=512, dropout_rate=dropout_rate)
        
        # Fully connected layers after concatenation
        self.fc1 = nn.Linear(512 * 3, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x1, x2, x3):
        # Split input into three separate branches
        # x1, x2, x3 = x  # Expect x to be a tuple of (image1, image2, image3)
        
        # Process each branch
        out1 = self.branch1(x1)
        out2 = self.branch2(x2)
        out3 = self.branch3(x3)
        
        # Concatenate the outputs from the three branches
        combined = torch.cat((out1, out2, out3), dim=1)
        
        # Pass through the final fully connected layers
        combined = self.fc1(combined)
        combined = nn.ReLU()(combined)
        combined = self.fc2(combined)
        
        return combined

def GetModel(num_classes=1, dropout_rate=0.5) -> nn.Module:
    return SiameseNetwork(num_classes=num_classes, dropout_rate=dropout_rate)


In [27]:
oModel = GetModel()

oModel = oModel.to(TORCH_DEVICE) #<! Transfer model to device
# torchinfo.summary(oModel, train_images.shape, device = TORCH_DEVICE)

train_images = (train_images[0], train_images[1], train_images[2])
# Pass the tuple of tensors as input to `torchinfo.summary`
torchinfo.summary(oModel, input_data=train_images, device=TORCH_DEVICE)

Layer (type:depth-idx)                   Output Shape              Param #
SiameseNetwork                           [32, 1]                   --
├─CustomConvNetWithResiduals: 1-1        [32, 512]                 --
│    └─Conv2d: 2-1                       [32, 64, 112, 112]        3,200
│    └─BatchNorm2d: 2-2                  [32, 64, 112, 112]        128
│    └─ReLU: 2-3                         [32, 64, 112, 112]        --
│    └─Conv2d: 2-4                       [32, 128, 56, 56]         204,928
│    └─BatchNorm2d: 2-5                  [32, 128, 56, 56]         256
│    └─ReLU: 2-6                         [32, 128, 56, 56]         --
│    └─Conv2d: 2-7                       [32, 256, 28, 28]         295,168
│    └─BatchNorm2d: 2-8                  [32, 256, 28, 28]         512
│    └─ReLU: 2-9                         [32, 256, 28, 28]         --
│    └─Dropout: 2-10                     [32, 256, 28, 28]         --
│    └─MaxPool2d: 2-11                   [32, 256, 14, 14]         --

In [28]:
# Logger 
# Wrapper of TensorBoard's `SummaryWriter` with index for iteration and epoch.

class TBLogger():
    def __init__( self, logDir: Optional[str] = None ) -> None:

        self.oTBWriter  = SummaryWriter(log_dir = logDir)
        self.iiEpcoh    = 0
        self.iiItr      = 0
        
        pass

    def close( self ) -> None:

        self.oTBWriter.close()

In [29]:
#Score function 

def Score(mScore: np.ndarray, vY: np.ndarray ) -> np.float64:
    
    preds = mScore >= 0.5
    valAcc = (preds.eq(vY).sum())/len(vY)

    return valAcc

#Loss function 

# Fix unbalance images

# # Compute manually
# train_labels = train_labels.long().flatten()          # Convert to integers (int64) and convert to 1 dim
# class_counts = torch.bincount(train_labels)
# total_samples = len(train_labels)
# class_weights = total_samples / (2.0 * class_counts)  # Calculate class weights (inversely proportional to class frequency)
# pos_weight = torch.tensor([class_weights[1]]).to(TORCH_DEVICE)
# hL = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# # Compute with function: compute_class_weight
# train_labels_cpu = train_labels.cpu().numpy() #need to pass to numpy from torch for compute class function 
# class_weights = compute_class_weight('balanced', classes=np.unique(train_labels_cpu), y=train_labels_cpu.flatten())
# class_weights = torch.tensor(class_weights, dtype=torch.float)

hL=nn.BCEWithLogitsLoss()#(pos_weight=class_weights[1])

# # Calculate the number of positive and negative samples -POS WEIGHT -got lower score 65
# num_pos = (train_labels_cpu == 1).sum()
# num_neg = (train_labels_cpu == 0).sum()
# pos_weight = num_neg / num_pos
# pos_weight = torch.tensor([pos_weight], dtype=torch.float).to(TORCH_DEVICE)
# hL = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

hL=hL.to(TORCH_DEVICE)


TENSOR_BOARD_BASE   = 'TB'

nEpochs=20

In [30]:
# # kaiming Weight initialization function
# def initialize_weights_he(model):
#     for layer in model.modules():
#         if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
#             nn.init.kaiming_uniform_(layer.weight, nonlinearity='leaky_relu')
#             if layer.bias is not None:
#                 nn.init.zeros_(layer.bias)


# # Apply weight initialization
# initialize_weights_he(oModel)


# # Xavier initialization function
# def initialize_weights_xavier(model):
#     for layer in model.modules():
#         if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
#             nn.init.xavier_uniform_(layer.weight)  # Use xavier_normal_ if you prefer normal distribution
#             if layer.bias is not None:
#                 nn.init.zeros_(layer.bias)

# # Apply weight initialization
# initialize_weights_xavier(oModel)

In [31]:
oModel = oModel.to(TORCH_DEVICE)

# Initial optimizer and scheduler
learnRate = 4e-3
oOpt = torch.optim.AdamW(oModel.parameters(), lr=6e-4, betas=(0.9, 0.99), weight_decay=1e-3)
oScd = torch.optim.lr_scheduler.OneCycleLR(oOpt, max_lr=learnRate, total_steps=nEpochs * len(train_loader))

# TensorBoard logger setup
oTBLogger = TBLogger(logDir=os.path.join(TENSOR_BOARD_BASE, 'Model_SiameseNetwork'))


# Train the model
oRunModel, history = Train_model(model=oModel, train_loader=train_loader, val_loader=test_loader, criterion=hL, optimizer=oOpt, scheduler=oScd, num_epochs=nEpochs, device=TORCH_DEVICE, is_binary=True, save_metric='f1', oTBLogger=None)


Train Epoch 1/20 - Batch 6948/7050 - Loss: 0.7032 - Gradient Norm: 0.2324

KeyboardInterrupt: 

In [28]:

# # Load the best model checkpoint
# checkpoint = torch.load('BestModel.pt', weights_only=False)  # Make sure 'weights_only=False' since you're loading other states like optimizer, etc.
# model = GetModel(num_classes=1)  # Initialize the model
# model.load_state_dict(checkpoint['Model'])  # Load the model's state dictionary

# # Switch the model to evaluation mode
# model.eval()

# # Example input tensor for extracting features (batch_size=1, channels=3, height=224, width=224)
# input_tensor = torch.randn(1, 3, 224, 224)

# # Extract features using the `extract_features` method
# with torch.no_grad():  # Disable gradients for faster inference
#     features = model.extract_features(input_tensor)

# # Print the shape of the extracted features
# print(f"Extracted Features Shape: {features.shape}")


In [29]:
# def visualize_feature_maps(feature_maps, num_feature_maps_to_show=5):
#     # Loop over feature maps from each layer
#     for i, fmap in enumerate(feature_maps):
#         fmap = fmap.detach().cpu().numpy()  # Convert to numpy array and detach from computation graph
        
#         # Take a few channels to display
#         num_channels = fmap.shape[1]
#         print(f"Layer {i} has {num_channels} channels.")
        
#         # Display a few channels
#         for j in range(min(num_feature_maps_to_show, num_channels)):
#             plt.imshow(fmap[0, j], cmap='viridis')
#             plt.title(f"Feature map from layer {i}, channel {j}")
#             plt.axis('off')
#             plt.show()

# # Initialize and load the model
# model = GetModel(num_classes=1)
# model.eval()

# # Example input tensor
# input_tensor = torch.randn(1, 3, 224, 224)

# # Extract feature maps
# with torch.no_grad():
#     feature_maps = model(input_tensor, return_feature_maps=True)

# # Visualize feature maps
# visualize_feature_maps(feature_maps, num_feature_maps_to_show=5)
